In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np

# Triangular fuzzy number class
class TriangularFuzzyNumber:
    def __init__(self, l, m, u):
        self.l = l  # lower
        self.m = m  # middle
        self.u = u  # upper

    def __add__(self, other):
        return TriangularFuzzyNumber(self.l + other.l, self.m + other.m, self.u + other.u)

    def __mul__(self, other):
        return TriangularFuzzyNumber(self.l * other.l, self.m * other.m, self.u * other.u)

    def defuzzify(self):
        return (self.l + self.m + self.u) / 3

    def reciprocal(self):
        return TriangularFuzzyNumber(1/self.u, 1/self.m, 1/self.l)

    def __truediv__(self, other):
        return self * other.reciprocal()

# Fuzzy comparison matrix (example: 4 stakeholders)
def create_fuzzy_matrix(values):
    n = len(values)
    matrix = [[None for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if i == j:
                matrix[i][j] = TriangularFuzzyNumber(1, 1, 1)
            elif i < j:
                matrix[i][j] = values[i][j]
                matrix[j][i] = values[i][j].reciprocal()
    return matrix

# Normalize fuzzy matrix and compute priority weights
def fuzzy_ahp_weights(matrix):
    n = len(matrix)
    col_sums = [TriangularFuzzyNumber(0, 0, 0) for _ in range(n)]
    for j in range(n):
        for i in range(n):
            col_sums[j] += matrix[i][j]

    norm_matrix = [[matrix[i][j] / col_sums[j] for j in range(n)] for i in range(n)]
    row_avg = []
    for i in range(n):
        row_sum = TriangularFuzzyNumber(0, 0, 0)
        for j in range(n):
            row_sum += norm_matrix[i][j]
        row_avg.append(row_sum.defuzzify() / n)
    return np.array(row_avg)

# Multiply weights and relationship values (simple linear QFD step)
def propagate_importance(requirements_weights, relationships_matrix):
    return np.dot(requirements_weights, relationships_matrix)

# Main flow
if __name__ == "__main__":
    # Step 2: Fuzzy AHP for stakeholder weights
    finance_vs_logistics = TriangularFuzzyNumber(1/3, 1/2, 1)  # example values
    finance_vs_mfg = TriangularFuzzyNumber(1, 2, 3)
    finance_vs_marketing = TriangularFuzzyNumber(2, 3, 4)
    logistics_vs_mfg = TriangularFuzzyNumber(2, 3, 4)
    logistics_vs_marketing = TriangularFuzzyNumber(3, 4, 5)
    mfg_vs_marketing = TriangularFuzzyNumber(1/2, 1, 2)

    stakeholder_matrix = create_fuzzy_matrix([
        [None, finance_vs_logistics, finance_vs_mfg, finance_vs_marketing],
        [None, None, logistics_vs_mfg, logistics_vs_marketing],
        [None, None, None, mfg_vs_marketing],
        [None, None, None, None]
    ])

    stakeholder_weights = fuzzy_ahp_weights(stakeholder_matrix)
    print("Stakeholder Weights:", stakeholder_weights)

    # Step 5: Stakeholder requirement importance (example values)
    rel_weights = np.array([
        [0.4, 0.3, 0.2, 0.1],  # Req1
        [0.1, 0.6, 0.2, 0.1],  # Req2
        [0.3, 0.1, 0.5, 0.1],  # Req3
    ])  # shape (requirements, stakeholders)

    req_weights = propagate_importance(stakeholder_weights, rel_weights.T)
    print("Requirement Weights:", req_weights)

    # Step 9: Evaluating factor weights based on requirements
    eval_rel = np.array([
        [0.6, 0.3, 0.1],  # Eval1 from Req1,2,3
        [0.3, 0.4, 0.3],  # Eval2
        [0.2, 0.3, 0.5],  # Eval3
    ])  # shape (factors, requirements)

    factor_weights = propagate_importance(req_weights, eval_rel.T)
    print("Evaluating Factor Weights:", factor_weights)

    # Step 13: Final 3PL scores (example matrix: alternatives x factors)
    alt_scores = np.array([
        [0.7, 0.6, 0.4],  # A1
        [0.5, 0.7, 0.3],  # A2
        [0.8, 0.9, 0.7],  # A3
        [0.6, 0.4, 0.5],  # A4
    ])

    final_scores = np.dot(alt_scores, factor_weights)
    print("3PL Final Scores:", final_scores)

    best_idx = np.argmax(final_scores)
    print(f"Best 3PL provider: A{best_idx+1}")


Stakeholder Weights: [0.32632873 0.51997267 0.16935802 0.14075465]
Requirement Weights: [0.33447036 0.39256354 0.24865036]
Evaluating Factor Weights: [0.34331632 0.33196163 0.30898831]
3PL Final Scores: [0.56309373 0.4967278  0.78971034 0.4932686 ]
Best 3PL provider: A3


In [3]:
import numpy as np
from itertools import combinations

# ------------------ Triangular Fuzzy Number ------------------ #
class TFN:
    def __init__(self, l, m, u):
        self.l = l
        self.m = m
        self.u = u

    def __add__(self, other):
        return TFN(self.l + other.l, self.m + other.m, self.u + other.u)

    def __truediv__(self, other):
        return TFN(self.l / other.u, self.m / other.m, self.u / other.l)

    def defuzzify(self):
        return (self.l + self.m + self.u) / 3

    def reciprocal(self):
        return TFN(1/self.u, 1/self.m, 1/self.l)

    def __repr__(self):
        return f"({self.l:.2f}, {self.m:.2f}, {self.u:.2f})"

# ------------------ Fuzzy AHP Core ------------------ #
class FuzzyAHP:
    def __init__(self, n_items, pairwise_values):
        self.n = n_items
        self.matrix = self._build_matrix(pairwise_values)

    def _build_matrix(self, pairwise_values):
        mat = [[None] * self.n for _ in range(self.n)]
        for i in range(self.n):
            mat[i][i] = TFN(1, 1, 1)
        for (i, j), val in pairwise_values.items():
            mat[i][j] = val
            mat[j][i] = val.reciprocal()
        return mat

    def _normalize(self):
        col_sum = [TFN(0, 0, 0) for _ in range(self.n)]
        for j in range(self.n):
            for i in range(self.n):
                col_sum[j] += self.matrix[i][j]

        norm = [[self.matrix[i][j] / col_sum[j] for j in range(self.n)] for i in range(self.n)]
        return norm

    def compute_weights(self):
        norm = self._normalize()
        row_avg = [TFN(0, 0, 0) for _ in range(self.n)]
        for i in range(self.n):
            for j in range(self.n):
                row_avg[i] += norm[i][j]
            row_avg[i] = row_avg[i].defuzzify() / self.n
        return np.array(row_avg)

# ------------------ QFD Layer Propagation ------------------ #
def qfd_propagate(source_weights, relationship_matrix):
    return np.dot(source_weights, relationship_matrix)

# ------------------ Example Use-Case ------------------ #
if __name__ == "__main__":
    # Step 1: Stakeholders
    stakeholders = ["Finance", "Logistics", "Manufacturing", "Marketing"]
    stakeholder_pairs = {
        (0, 1): TFN(1/3, 1/2, 1),
        (0, 2): TFN(2, 3, 4),
        (0, 3): TFN(2, 3, 4),
        (1, 2): TFN(1, 2, 3),
        (1, 3): TFN(3, 4, 5),
        (2, 3): TFN(1/2, 1, 2),
    }
    stakeholder_ahp = FuzzyAHP(len(stakeholders), stakeholder_pairs)
    stakeholder_weights = stakeholder_ahp.compute_weights()
    print("\nStakeholder Weights:")
    for name, w in zip(stakeholders, stakeholder_weights):
        print(f"  {name}: {w:.4f}")

    # Step 2: Stakeholder Requirements
    requirements = [
        "Reduce total logistics costs", "Reduce cycle time", "Assure quality",
        "Provide customized services", "Increase customer satisfaction",
        "Advanced tech", "Timely advice", "Problem resolution"
    ]

    # Matrix: (stakeholders x requirements)
    rel_stake_req = np.array([
        [0.3, 0.2, 0.2, 0.1, 0.05, 0.05, 0.05, 0.05],   # Finance
        [0.1, 0.3, 0.2, 0.1, 0.1, 0.05, 0.1, 0.05],     # Logistics
        [0.2, 0.2, 0.2, 0.2, 0.1, 0.1, 0, 0],           # Manufacturing
        [0.05, 0.1, 0.15, 0.2, 0.3, 0.05, 0.1, 0.05],   # Marketing
    ])

    req_weights = qfd_propagate(stakeholder_weights, rel_stake_req)
    print("\nRequirement Weights:")
    for name, w in zip(requirements, req_weights):
        print(f"  {name}: {w:.4f}")

    # Step 3: Evaluating Factors (20 total, simplified here to 6)
    eval_factors = [
        "Cost efficiency", "Delivery speed", "Service flexibility",
        "Product quality", "Technology use", "Risk management"
    ]

    rel_req_eval = np.array([
        [0.4, 0.2, 0.1, 0.1, 0.1, 0.1],   # Reduce total logistics costs
        [0.1, 0.4, 0.2, 0.1, 0.1, 0.1],   # Reduce cycle time
        [0.1, 0.1, 0.2, 0.4, 0.1, 0.1],   # Assure quality
        [0.05, 0.1, 0.5, 0.1, 0.15, 0.1], # Customization
        [0.1, 0.2, 0.1, 0.3, 0.2, 0.1],   # Satisfaction
        [0.1, 0.1, 0.1, 0.1, 0.5, 0.1],   # Tech
        [0.05, 0.05, 0.1, 0.1, 0.2, 0.5], # Advice
        [0.1, 0.05, 0.1, 0.1, 0.1, 0.55], # Problem solving
    ])

    factor_weights = qfd_propagate(req_weights, rel_req_eval)
    print("\nEvaluating Factor Weights:")
    for name, w in zip(eval_factors, factor_weights):
        print(f"  {name}: {w:.4f}")

    # Step 4: 3PL Alternatives
    alternatives = ["A1", "A2", "A3", "A4"]
    eval_scores = np.array([
        [0.6, 0.7, 0.4, 0.5, 0.6, 0.4],   # A1
        [0.5, 0.5, 0.6, 0.4, 0.6, 0.3],   # A2
        [0.8, 0.9, 0.7, 0.8, 0.9, 0.7],   # A3
        [0.4, 0.6, 0.5, 0.5, 0.5, 0.5],   # A4
    ])

    final_scores = np.dot(eval_scores, factor_weights)
    print("\nFinal 3PL Scores:")
    for name, score in zip(alternatives, final_scores):
        print(f"  {name}: {score:.4f}")

    best = alternatives[np.argmax(final_scores)]
    print(f"\n✅ Best 3PL Provider: {best}")



Stakeholder Weights:
  Finance: 0.3636
  Logistics: 0.4785
  Manufacturing: 0.1770
  Marketing: 0.1397

Requirement Weights:
  Reduce total logistics costs: 0.1993
  Reduce cycle time: 0.2656
  Assure quality: 0.2248
  Provide customized services: 0.1476
  Increase customer satisfaction: 0.1256
  Advanced tech: 0.0668
  Timely advice: 0.0800
  Problem resolution: 0.0491

Evaluating Factor Weights:
  Cost efficiency: 0.1643
  Delivery speed: 0.2216
  Service flexibility: 0.2239
  Product quality: 0.2084
  Technology use: 0.1705
  Risk management: 0.1700

Final 3PL Scores:
  A1: 0.6178
  A2: 0.5640
  A3: 0.9269
  A4: 0.5851

✅ Best 3PL Provider: A3
